<a href="https://colab.research.google.com/github/albertlis/Plant-Disease-Recognition/blob/main/notebooks/classifier/Conv_EfficientNetB0_predictions_3_layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import tensorflow as tf

drive.mount("/content/drive")
!unzip -q /content/drive/My\ Drive/PDR/Data/original.zip -d /content
!rm -r sample_data

Mounted at /content/drive


Constants

In [5]:
img_shape = (224, 224, 3)
e_net_out_shape = (7, 7, 1280)
nr_of_imgs = 49940
nr_of_val_imgs = 3862
batch_size = 64
nr_of_classes = 39
train_path = './original/train'
val_path = './original/val'

Get predictions from Convolutional part to fast train classifier

In [3]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

e_net = EfficientNetB0(include_top=False, weights="imagenet", input_shape=img_shape)

def make_conv_predictions():
    gen = ImageDataGenerator(rotation_range=45, horizontal_flip=True, vertical_flip=True, rescale=1/255)
    datagen = gen.flow_from_directory(train_path, target_size=img_shape[:2], batch_size=batch_size, class_mode='categorical')
    val_datagen = gen.flow_from_directory(val_path, target_size=img_shape[:2], batch_size=batch_size, class_mode='categorical')

    imgs = np.lib.format.open_memmap('imgs.npy', dtype='float32', mode='w+', shape=((nr_of_imgs,) + e_net_out_shape))
    labels = np.lib.format.open_memmap('labels.npy', dtype='uint8', mode='w+', shape=(nr_of_imgs, nr_of_classes))
    val_imgs = np.lib.format.open_memmap('val_imgs.npy', dtype='float32', mode='w+', shape=((nr_of_val_imgs,) + e_net_out_shape))
    val_labels = np.lib.format.open_memmap('val_labels.npy', dtype='uint8', mode='w+', shape=(nr_of_val_imgs, nr_of_classes))

    for i, (imgs_batch, labels_batch) in enumerate(datagen):
        count = i * batch_size
        line = ' '
        if not i % 20 and i != 0:
            line = '\n'
        print(f'%5d{line}' %(count), end='')
        if count > nr_of_imgs:
            break
        predictions = e_net.predict(imgs_batch)
        imgs[count : count + batch_size] = predictions
        labels[count : count + batch_size] = labels_batch
    print()

    for i, (imgs_batch, labels_batch) in enumerate(val_datagen):
        count = i * batch_size
        line = ' '
        if not i % 20 and i != 0:
            line = '\n'
        print(f'%5d{line}' %(count), end='')
        if count > nr_of_val_imgs:
            break
        predictions = e_net.predict(imgs_batch)
        val_imgs[count : count + batch_size] = predictions
        val_labels[count : count + batch_size] = labels_batch
    print()
    
make_conv_predictions()

16711680/16705208 [==============================] - 0s 0us/step
Found 49940 images belonging to 39 classes.
Found 3862 images belonging to 39 classes.
    0   512  1024  1536  2048  2560  3072  3584  4096  4608  5120  5632  6144  6656  7168  7680  8192  8704  9216  9728 10240
10752 11264 11776 12288 12800 13312 13824 14336 14848 15360 15872 16384 16896 17408 17920 18432 18944 19456 19968 20480
20992 21504 22016 22528 23040 23552 24064 24576 25088 25600 26112 26624 27136 27648 28160 28672 29184 29696 30208 30720
31232 31744 32256 32768 33280 33792 34304 34816 35328 35840 36352 36864 37376 37888 38400 38912 39424 39936 40448 40960
41472 41984 42496 43008 43520 44032 44544 45056 45568 46080 46592 47104 47616 48128 48640 49152 49664 50176 
    0   512  1024  1536  2048  2560  3072  3584  4096 


In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, InputLayer, BatchNormalization, Dropout
import tensorflow.keras.callbacks as clb
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import backend as K 
import numpy as np
import random

def build_model():
    model = Sequential()
    
    model.add(InputLayer(input_shape=e_net_out_shape))
    model.add(Flatten())
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(867, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(453, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(39, activation='softmax'))

    model.compile(optimizer=Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=['acc'])
    return model

def np_array_memmap_gen(feat_path, label_path, batch_size=128, shuffle_array=True):
    while 1:
        x = np.load(feat_path, mmap_mode='r')
        y = np.load(label_path, mmap_mode='r')
        lst = [i for i in range(x.shape[0])]

        if shuffle_array:
            random.shuffle(lst)

        iters = len(lst) // batch_size + 1

        for i in range(iters):
            start = i * batch_size
            end = (i + 1) * batch_size
            yield (x[lst[start : end]], y[lst[start : end]])

callbacks = [
            clb.ReduceLROnPlateau(monitor='val_acc', factor=0.1, min_lr=1e-7, patience=2, verbose=1),
            clb.EarlyStopping(monitor='val_acc', patience=4, verbose=1),
            clb.ModelCheckpoint(monitor='val_acc', filepath='/content/drive/My Drive/PDR/Results/models/external_fast_model.h5',
                                save_best_only=True, verbose=1)
            ]

train_gen = np_array_memmap_gen('imgs.npy', 'labels.npy', batch_size=batch_size)
val_gen = np_array_memmap_gen('val_imgs.npy', 'val_labels.npy', batch_size=batch_size)

train_steps = nr_of_imgs // batch_size + 1
val_steps = nr_of_val_imgs // batch_size + 1

model = build_model()

history = model.fit(train_gen, epochs=50, validation_data=val_gen, callbacks=callbacks, verbose=1, 
                    steps_per_epoch=train_steps, validation_steps=val_steps)

Epoch 1/50
781/781 [==============================] - 643s 823ms/step - loss: 2.8159 - acc: 0.2740 - val_loss: 2.0163 - val_acc: 0.3791

Epoch 00001: val_acc improved from -inf to 0.37908, saving model to /content/drive/My Drive/PDR/Results/models/external_fast_model.h5
Epoch 2/50
781/781 [==============================] - 675s 866ms/step - loss: 1.7636 - acc: 0.4706 - val_loss: 1.7905 - val_acc: 0.4858

Epoch 00002: val_acc improved from 0.37908 to 0.48576, saving model to /content/drive/My Drive/PDR/Results/models/external_fast_model.h5
Epoch 3/50
781/781 [==============================] - 674s 864ms/step - loss: 1.5743 - acc: 0.5210 - val_loss: 4.6526 - val_acc: 0.4143

Epoch 00003: val_acc did not improve from 0.48576
Epoch 4/50
781/781 [==============================] - 665s 852ms/step - loss: 1.4361 - acc: 0.5629 - val_loss: 4.0204 - val_acc: 0.2683

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.0009999999776482583.

Epoch 00004: val_acc did not improve from 0.48576
